In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 
import spacy
import re
from functools import reduce
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import ensemble
from sklearn.model_selection import train_test_split

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


height has been deprecated.



In [2]:
# Data obtained from Figure Eight.  Contains economic news articles that were categorized as either having information on the US economy or not.
data = pd.read_csv("Economic-News.csv",encoding = "ISO-8859-1")
#link: https://www.figure-eight.com/wp-content/uploads/2016/03/Full-Economic-News-DFE-839861.csv

In [3]:
# See a few rows of the data
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [4]:
# See data shape
data.shape

(8000, 15)

In [5]:
# Check for imbalance
data["relevance"].value_counts()


no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [6]:
#Separate data into relevant and not relevant for parsing
relevant = data[data["relevance"] == "yes"]
not_relevant = data[data["relevance"] == "no"]
relevant = relevant.iloc[0:500,]
not_relevant = not_relevant.iloc[0:500,]
yes_text = relevant["text"]
no_text = not_relevant["text"]

In [7]:
#Create list to see samples of data to determine what cleaning is needed
text= data["text"]
relevance = data["relevance"]
text_list= (1,100,500,1000,1500,2250,2500,3000,3350,3400,4500)

In [8]:
# See text to determine cleaning needed.
for headline in text_list:
    print (text[headline][:50]) #Remove slice to see full data while determining what cleaning is needed

The Wall Street Journal Online</br></br>The Mornin
J.P. Morgan Chase & Co. reached a tentative deal t
Five large banks reported improved second-quarter 
SANTA CLARA, Calif. -- George Bush played down the
It's no secret that the tea party faithful regard 
AMSTERDAM--Dutch retailer Royal Ahold NV said Wedn
Behind every good rally these days is a good deal 
South Korea's efforts to engineer an economic reco
PLATTSBURGH, N.Y. -- The rise of conservative "tea
{First of a Series}</br></br>W.J. Turner L. Cobden
Regan, in his strongest language to date, blamed b


In [9]:
# Clean the text
def text_cleaner(text):
    text = text.apply(lambda x:re.sub("</br></br>", "", x))
    text = text.apply(lambda x:re.sub("---", "", x))
    text = text.apply(lambda x:re.sub("--", "", x))
    text = text.apply(lambda x:re.sub("<", "", x))
    text = text.apply(lambda x:re.sub(";", "", x))
    text = text.apply(lambda x:re.sub(";", "", x))
    text = text.apply(lambda x:re.sub("ÛÏ", "", x))
    text = text.apply(lambda x:re.sub("Ûª", "", x))
    text = text.apply(lambda x:re.sub("Û", "", x))
    text = text.apply(lambda x:re.sub("ûó", "", x))
    text = text.apply(lambda x:re.sub("ÛÓ", "", x))
    text = text.apply(lambda x:re.sub("\x89ûó", "", x))
    text = text.apply(lambda x:re.sub("x89ûó", "", x))
    text = text.apply(lambda x:re.sub("\x89ðê", "", x))  
    text = text.apply(lambda x:re.sub("Author:", "", x))
    text = text.apply(lambda x:re.sub("By:", "", x))
    return text
cleaned_text=text_cleaner(text)
yes_cleaned = text_cleaner(yes_text)
no_cleaned = text_cleaner(no_text)
    

In [32]:
# See cleaned text and check if additional cleaning needed
for headline in text_list:
    print (cleaned_text[headline][:50]) #Remove slice if needed to see full data

The Wall Street Journal OnlineThe Morning Brief, a
J.P. Morgan Chase & Co. reached a tentative deal t
Five large banks reported improved second-quarter 
SANTA CLARA, Calif.  George Bush played down the n
It's no secret that the tea party faithful regard 
AMSTERDAMDutch retailer Royal Ahold NV said Wednes
Behind every good rally these days is a good deal 
South Korea's efforts to engineer an economic reco
PLATTSBURGH, N.Y.  The rise of conservative "tea p
{First of a Series}W.J. Turner L. Cobden, a top mu
Regan, in his strongest language to date, blamed b


In [11]:
#Combine rows into one string for Spacy parsing.
yes_text_combined = reduce(lambda x, y: x+y, yes_cleaned)
yes_text_combined = yes_text_combined.lower()


no_text_combined = reduce(lambda x, y: x+y, no_cleaned)
no_text_combined = no_text_combined.lower()


In [12]:
#Load English Spacy
nlp = spacy.load("en")

In [13]:
#Increase max length for nlp.  Then run Spacy parsing on relevant and not relevant
nlp.max_length = 11000000
yes_nlp = nlp(yes_text_combined)
no_nlp = nlp(no_text_combined)

In [14]:
#Convert parsed text to sentences
yes_sents = [[sent,"Relevant"] for sent in yes_nlp.sents]
no_sents = [[sent,"Not_Relevant"] for sent in no_nlp.sents]
sentences = pd.DataFrame(yes_sents + no_sents)
nlp_text =sentences
sentences.head(10)

,0,1
0,"(new, york, , yields, on, most, certificates,...",Relevant
1,"(two, banks, in, the, banxquote, survey, ,, ci...",Relevant
2,"(against, the, yen, ,, the, dollar, was, tradi...",Relevant
3,"(against, the, swiss, franc, ,, the, dollar, w...",Relevant
4,"(17.weighing, on, the, downside, were, alcoa, ...",Relevant
5,"(hewlett, -, packard, dropped, 21, cents, ,, o...",Relevant
6,"(the, decline, continued, in, new, york, tradi...",Relevant
7,"(lower, bond, prices, translate, into, higher,...",Relevant
8,"(the, bond, market, later, closed, little, -, ...",Relevant
9,"(,, new, york.in, late, new, york, trading, ,,...",Relevant


In [15]:
# Word counter function
def word_counter(text, include_stop=True):
        words=[]
        for token in text:
            if not token.is_punct and (not token.is_stop or include_stop):
                words.append(token.text)
        return Counter(words)

In [16]:
# Word frequency for relevant text
word_frequency_relevant = word_counter(yes_nlp, include_stop=False).most_common(20)
print(word_frequency_relevant)

[("'s", 1008), ('$', 593), ('said', 517), ('market', 475), ('u.s', 471), ('year', 430), (' ', 406), ('prices', 375), ('economy', 369), ('rate', 355), ('rates', 345), ('economic', 337), ('interest', 326), ('fed', 305), ('new', 303), ('stock', 278), ('federal', 271), ('dollar', 265), ('quarter', 259), ('growth', 250)]


In [17]:
# Word frequency for non-relevant
word_frequency_not_relevant = word_counter(no_nlp, include_stop=False).most_common(20)
print(word_frequency_not_relevant)

[("'s", 1157), ('$', 635), ('u.s', 454), (' ', 446), ('market', 441), ('said', 441), ('year', 369), ("n't", 357), ('stock', 339), ('new', 307), ('rates', 260), ('investors', 240), ('billion', 233), ('interest', 233), ('economic', 229), ('economy', 221), ('federal', 217), ('rate', 215), ('million', 214), ('mr', 212)]


In [18]:
# Function to count lemmas
def lemma_frequencies(text, include_stop=True):
    lemmas = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            lemmas.append(token.lemma_)
    return Counter(lemmas)

In [19]:
#See most common lemmas for relevant text
text_lemma_freq_relevant = lemma_frequencies(yes_nlp, include_stop=False).most_common(10)
print(text_lemma_freq_relevant)

[("'s", 875), ('rate', 700), ('say', 607), ('market', 596), ('$', 593), ('year', 550), ('price', 510), ('stock', 460), ('rise', 438), ('u.s', 414)]


In [20]:
#See most common lemmas
text_lemma_freq_not_relevant = lemma_frequencies(no_nlp, include_stop=False).most_common(10)
print(text_lemma_freq_not_relevant)

[("'s", 1016), ('$', 635), ('say', 601), ('market', 579), ('year', 505), ('stock', 483), ('rate', 475), (' ', 446), ('u.s', 379), ('not', 357)]


In [21]:
#Bag of words features
def bag_of_words(text):
    allwords = [token.lemma_ for token in text if not token.is_punct and not token.is_stop]
    return [item[0] for item in Counter(allwords).most_common(2000)]

def bow_features(sentences, common_words):
    bow_df = pd.DataFrame(columns=common_words)
    bow_df["text"] = sentences[0]
    bow_df["relevance"] = sentences[1]
    bow_df.loc[:,common_words] = 0
    for i, sentence in enumerate(bow_df["text"]):
        words = [token.lemma_ for token in sentence if (not token.is_punct and not token.is_stop and token.lemma_ in common_words)]
        for word in words:
            bow_df.loc[i, word] +=1
        if i % 500 == 0:
            print("Processing row {}".format(i))
    return bow_df

In [22]:
# Run bag of words on relevant and not relevant sets.  Then combine.
yes_words = bag_of_words(yes_nlp)
no_words = bag_of_words(no_nlp)
common_words = set(yes_words + no_words)

In [23]:
# Add words to dataframe then count for each row of text
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500


,somewhat,part,campaign,disappear,productive,index,launch,health,goal,turn,25,modest,initially,church,1986,workforce,witness,seven,selling,compound,toll,wholesale,alike,outstanding,3.4,daniel,bed,a.,presidential,exposure,status,hire,war,introduce,fail,broker,reduction,insist,recovery.the,2.9,inquiry,necessarily,safe,unusual,familiar,5.4,recovery,reject,meeting,treasury,1970,pause,uneven,intensify,airline,unemployment,link,thin,solvency,week.the,line,earlier.the,pound,seasonal,liberal,overnight,output,ignore,previous,tokyo,passage,slower,wal,publishing,administration,turmoil,banxquote,sector,expenditure,joseph,10-year,example,operating,deadline,making,apparel,lag,increasingly,friday,especially,home,hearing,agreement,calm,crist,arm,watcher,shake,ge,specific,curve,edward,ratio,commission,bellwether,indian,sour,cause,estate,share,comment,article,poor,take,slowdown,digit,lift,fannie,mystery,kudlow,woe,hong,free,calif,manufacturing,1.2,j.p,board,"200,000",manufacturer,relative,bad,despite,change,december,couple,anytime,anxiety,hop,jobless,science,threaten,janet,drift,need,india,retailer,certainly,testing,spot,transform,debate,oil,analyze,worry,video,shape,decent,weekly,michael,recently,38,survey,negative,journal,gate,seasonally,contribution,fiscal,retirement,northeast,28,obvious,ward,deal,canada,thomson,speculation,contract,fast,definitely,22,wall,platinum,implement,citigroup,2.5,1995,corner,chapter,holding,underscore,opposition,downbeat,core,insurer,service,reversal,marketing,half,pose,smooth,mark,human,extraordinary,ounce,reporter,snap,rush,historical,w.,proposal,approach,nature,rebound,escape,allege,upper,coalition,reflect,quantitative,gold,kick,37,"20,000",step,carrier,boston,bush,send,reinforce,coming,indicator,government,significantly,lender,competition,hurricane,knock,tiny,score,go,new,tighten,development,sterling,sizable,participation,round,forward,...,facility,principal,greenback,critic,6.3,lehman,revive,place,non,situation,fierce,benchmark,aug,sharply,money,speak,session,caution,drive,ahead,bell,victory,common,39,hang,trust,people,batter,3.1,chrysler,visit,germany,soon,oct,statistical,investigation,2,world,deposit,cloud,2010,m1,enormous,pia,far,course,separate,nominal,president,forecaster,mae,agenda,treat,fall,decision,1994,tight,picture,produce,narrow,east,liquidity,source,1.4,tool,im,subprime,minute,sponsor,relate,slump,creek,mix,legislative,category,brady,soft,train,mayor,october,decade,average,industry,optimistic,favor,benefit,guide,reluctance,bailout,weakness,institute,undermine,debut,flat,correct,be,4.8,lie,virtually,35,cheap,bureau,2001,squeeze,4,bull,tender,impose,bolster,count,ticket,age,access,small,24,internal,revenue,baker,publisher,asia,advancer,woman,demand,premium,own,steady,franc,1999,understand,consecutive,close,indicate,thailand,insurance,corp,schedule,greenspan,wave,mail,e,highlight,55,necessary,roger,feeling,track,politician,seriously,international,gear,basic,energy,crude,ceiling,restructuring,truck,sentiment,ian,variety,49,practice,month,emerge,shopper,client,publish,damage,3.2,southeast,chain,housing,pretty,clean,inc,invest,unchanged,system,anticipate,holiday,shy,anecdotal,investment,reach,conviction,earn,year,par,1981,well,middle,back,2.7,unprecedented,lackluster,alhadeff,stream,publication,hewlett,1988,trial,pickup,yen,spur,welcome,affect,reluctant,amount,rep,association,competitor,provision,chief,strategist,aggressively,1973,louis,regard,eighth,urge,adelphia,momentum,a.m.,sustain,14,d.,fraud,5.3,ease,median,college,deliver,restrain,declare,january,enthusiasm,anemic,evan,aide,frequency,improvement,thanksgiving,easy,600,tenure,maybe,40,employment,gop,text,relevance
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0

In [70]:
#Random forest model function
def random_forest_model(x_train,y_train,x_test,y_test, random_state=42):
    rfc = ensemble.RandomForestClassifier()
    model = rfc.fit(x_train, y_train)
    print('Training set score:', model.score(x_train, y_train))
    print('\nTest set score:', model.score(x_test, y_test))
    cross = cross_val_score(rfc,x_test, y_test, cv=3)
    print(cross)
    

In [71]:
# Bag of words train-test-split

y = word_counts['relevance']
x = np.array(word_counts.drop(['text','relevance'], 1))

x_train_bow, x_test_bow, y_train_bow, y_test_bow = train_test_split(x, y, test_size=0.4, random_state=0)

In [84]:
#Random forest model function
def random_forest_model(estimators,x_train,y_train,x_test,y_test, random_state=42):
    rfc = ensemble.RandomForestClassifier(n_estimators=estimators)
    model = rfc.fit(x_train, y_train)
    print('Training set score:', model.score(x_train, y_train))
    print('\nTest set score:', model.score(x_test, y_test))
    cross = cross_val_score(rfc,x_test, y_test, cv=3)
    print(cross)

In [85]:
# Run random forest on bag of words features
random_forest_model(10,x_train_bow, y_train_bow, x_test_bow, y_test_bow)

Training set score: 0.97812033814

Test set score: 0.596569724087
[ 0.58501119  0.60402685  0.60290828]


In [86]:
# Run random forest on bag of words features
random_forest_model(500,x_train_bow, y_train_bow, x_test_bow, y_test_bow)

Training set score: 0.99129786176

Test set score: 0.647278150634
[ 0.62527964  0.62751678  0.63534676]


In [97]:
# Run random forest on bag of words features
random_forest_model(1000,x_train_bow, y_train_bow, x_test_bow, y_test_bow)

Training set score: 0.99129786176

Test set score: 0.648769574944
[ 0.62527964  0.63422819  0.63534676]


In [38]:
#Create vectorizer with train-test-split

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', lowercase=True, use_idf=True, norm=u'l2', smooth_idf=True)

vector_x_train, vector_x_test, vector_y_train, vector_y_test = train_test_split(cleaned_text, relevance, test_size=0.4, random_state=0)

In [39]:
#Vectorizer fit and transform.
text_tfidf = vectorizer.fit_transform(cleaned_text)

x_train_tfidf, x_test_tfidf = train_test_split(text_tfidf, test_size=.4, random_state=42)

x_train_tfidf_csr = x_train_tfidf.tocsr()

n = x_train_tfidf_csr.shape[0]

tfidf_bypara = [{} for _ in range(0,n)]

terms = vectorizer.get_feature_names()

for i, j in zip(*x_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = x_train_tfidf_csr[i, j]


In [40]:
#SVD to reduce features created by LSA and vectorizer
svd = TruncatedSVD(1000)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
x_train_lsa = lsa.fit_transform(x_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(x_train_lsa,index=vector_x_train)

for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

Percent variance captured by all components: 58.8032116463
Component 0:
text
Although President Reagan said Thursday that he opposes extending special unemployment benefits for the long-term jobless because retraining is a better solution, he has asked Congress to rescind $119 million of the $222 million it had -approved for retraining dislocated ÐÊworkers in 1985.Office of Management and Budget spokesman Edwin L. Dale Jr. said yesterday that the full amount is not needed because money is left l over since the states werent able to use it effectively. But Jim Ellen-berger of the AFL-CIO's unemployment insurance department called the presidents position hypocritical.At his news conference Thursday night, the president was asked Û¢whether he supported legislation to extend a program that would provide up to 14 weeks of additional unemployment benefits to 325,000 workers who have exhausted their regular state benefits.300,000 new jobs a month and in part because he believes the place "f

Name: 0, dtype: float64
Component 1:
text
CONSUMER SPENDING jumped 0.9% in November, suggesting that Christmas shopping may be stronger than had been thought. The gain followed a 1.4% drop in October. November personal income rose 0.6%, the government said.The FDIC has begun selling pooled assets from failed banks to investment bankers for resale as securities. The action suggests the agency needs help in handling the record flow of assets from failing institutions.R.J. Reynolds isn't liable for the death of a longtime smoker of the company's cigarettes, a jury ruled in Santa Barbara, Calif. It was the first jury verdict in a new wave of product-liability suits against tobacco firms.                                                                                                                                                                                                                                                                                                                    

Name: 1, dtype: float64
Component 2:
text
A sharp increase in the number of unemployed persons in Washington's suburbs pushed the metropolitan-area unemployment rate to 4.0 percent in December, its highest level in six years, according to figures released yesterday.Unemployment in the District fell significantly, from 6.9 percent in November to 6.5 percent. But officials said the drop mostly reflected Christmas-season hiring, not a turnaround in the troubled economy.Christmastime hiring in the entire area was only about half what it has been over the last five years. From September to December 1990, 10,800 jobs were added in the retail trade, according to Richard Groner of the D.C. Department of Employment Services, which compiles the area statistics. Over the previous five years, the average increase in jobs during the same period was 21,000.In Maryland, which also released its unemployment rate yesterday, joblessness went up from 5.3 percent in November to 6.0 percent in December. Of

Name: 2, dtype: float64
Component 3:
text
Deflation knocked louder at the door of the U.S. economy as the Bridge Commodity Research Bureau index ended the day at a 12-year low.The index, composed of 17 major commodity prices, slipped early in the day below an August 1992 intraday low of 198.17  seen as a key benchmark  and dipped to 197.72 before settling down 1.65 at 198.12. That marked its lowest close since 1986. The move was the latest slip in the index's year-long 39.57-point fall from 237.69, signaling commodity markets' two-year bear market may be picking up steam rather than abating.The Goldman Sachs index of spot commodity prices also slipped 1.41 to 139.6070, its lowest level since 1977.Most industry observers don't believe commodity prices will recover any time soon. "It's falling like a stone," said Ray Murphy, president of RTM Management Inc., who helped Bridge/CRB create a CRB total-returns index tailored to investors like pension funds. "There's plenty of supply of every

Name: 3, dtype: float64
Component 4:
text
WASHINGTON  The Federal Reserve is likely, in light of November's strong employment figures, to decide to raise short-term interest rates at its policy meeting Dec. 14.Interviews with Fed governors and district presidents who make up the central bank's policy committee suggest that many of them believe higher interest rates are needed to cool down the economy and prevent an acceleration of inflation.The 463,000-job increase in payroll employment "was far bigger than anyone thought it was going to be," said Robert Black, president of the Federal Reserve Bank of Richmond. The latest numbers add to "anecdotal information from our district that the economy continues to be robust," agrees Cleveland Fed President Lee Hoskins. "There is some sign here that policy should continue to err on the side of containing inflation."But the Fed is unlikely to take decisive action before the Dec. 14 meeting unless it is confronted with a steep decline in the doll

Name: 4, dtype: float64


In [41]:
#Create x train dataframe with features for modeling
vector_x_train_transformed = pd.DataFrame(x_train_lsa)

#Transform x_test
x_test_lsa = lsa.transform(x_test_tfidf)

#Create dataframe with x test transformed features
vector_x_test_transformed = pd.DataFrame(x_test_lsa)

In [44]:
vector_x_train_transformed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0.311456,0.186323,0.104057,0.021605,0.055890,-0.008709,-0.095838,-0.006254,0.057284,-0.056736,0.113009,-0.041625,-0.062383,0.187888,-0.060860,0.020284,0.033974,-0.029529,0.069041,-0.044434,0.071750,-0.000282,0.009860,0.042830,-0.065079,-0.019954,0.037924,-0.012309,-0.007381,-0.028569,0.024100,-0.010929,0.001122,0.020142,-0.031176,0.004304,0.104560,-0.028530,-0.054034,0.070354,-0.039562,-0.036183,-0.018638,0.024511,-0.003032,0.012832,0.017615,-0.037168,-0.011519,0.008427,-0.007564,-0.006532,-0.040148,-0.019167,-0.027993,-0.036195,-0.042486,0.076353,-0.011527,-0.064344,-0.039963,0.034163,-0.044342,0.021094,-0.010213,0.006519,0.084156,-0.055876,-0.055204,0.000799,0.110446,-0.047510,-0.021597,-0.044042,0.028665,-0.006475,0.054399,0.019529,0.023851,-0.033764,-0.019187,-0.034015,-0.005977,-0.010280,-0.035062,0.034782,0.042442,0.018018,0.002017,-0.007490,-0.092513,0.014309,0.048590,-0.011174,-0.041442,-0.069797,-0.026903,-0.026687,0.017518,-0.020451,-0.044566,-0.059526,-0.014810,0.017883,0.011827,0.000193,0.029242,-0.052037,-0.052089,0.027469,0.007149,-0.024137,-0.026369,0.015979,0.057706,0.024248,-0.044561,-0.002282,0.018020,-0.022651,-0.011816,0.048409,0.050347,-0.007864,-0.028058,0.040548,0.013741,0.060819,0.013276,0.010207,0.010192,0.058794,0.009088,-0.029051,-0.005451,0.015284,-0.046318,-0.031128,0.010390,-0.030384,-0.015763,-0.047111,-0.002199,0.070943,-0.028571,0.071945,0.039055,-0.015586,0.024361,-0.051123,0.006666,-0.044176,-0.040946,-0.031611,0.012048,-0.099686,-0.013345,0.000800,-0.054161,-0.036677,0.002124,-0.006557,0.023712,-0.053463,0.002157,-0.070529,0.014318,-0.030630,-0.008073,-0.027713,-0.003737,0.021297,0.027508,-0.062899,0.017898,0.030075,0.030801,-0.012415,0.043412,-0.024927,0.025086,-0.011513,-0.062482,0.060263,0.024304,-0.021653,-0.006128,0.025976,-0.021966,0.055033,0.011026,-0.026732,-0.018066,-0.058192,0.022480,-0.027379,-0.045280,0.006288,-0.038730,0.025729,-0.001442,-0.080067,0.008062,-0.028958,-0.010071,-0.073148,0.024585,-0.023458,0.024969,0.032078,0.018033,0.094670,-0.012763,0.011307,-0.021780,0.047004,-0.004656,-0.022148,-0.068689,-0.006451,0.

In [42]:
# Random Forest on vectorizer features

random_forest_model(vector_x_train_transformed,vector_y_train,vector_x_test_transformed,vector_y_test)

Training set score: 0.973958333333

Test set score: 0.8196875
[ 0.80880975  0.81162137  0.81332083]


In [64]:

def vectorizer_model():

    #Create vectorizer with train-test-split

    vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', lowercase=True, use_idf=False, norm=u'l2', smooth_idf=True)

    vector_x_train, vector_x_test, vector_y_train, vector_y_test = train_test_split(cleaned_text, relevance, test_size=0.4, random_state=0)
    #Vectorizer fit and transform.
    text_tfidf = vectorizer.fit_transform(cleaned_text)

    x_train_tfidf, x_test_tfidf = train_test_split(text_tfidf, test_size=.4, random_state=42)

    x_train_tfidf_csr = x_train_tfidf.tocsr()

    n = x_train_tfidf_csr.shape[0]

    tfidf_bypara = [{} for _ in range(0,n)]

    terms = vectorizer.get_feature_names()

    for i, j in zip(*x_train_tfidf_csr.nonzero()):
        tfidf_bypara[i][terms[j]] = x_train_tfidf_csr[i, j]

        #SVD to reduce features created by LSA and vectorizer
    svd = TruncatedSVD(50)
    lsa = make_pipeline(svd, Normalizer(copy=False))
    # Run SVD on the training data, then project the training data.
    x_train_lsa = lsa.fit_transform(x_train_tfidf)

    variance_explained=svd.explained_variance_ratio_
    total_variance = variance_explained.sum()
    print("Percent variance captured by all components:",total_variance*100)

    #Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
    paras_by_component=pd.DataFrame(x_train_lsa,index=vector_x_train)

    #Create x train dataframe with features for modeling
    vector_x_train_transformed = pd.DataFrame(x_train_lsa)

    #Transform x_test
    x_test_lsa = lsa.transform(x_test_tfidf)

    #Create dataframe with x test transformed features
    vector_x_test_transformed = pd.DataFrame(x_test_lsa)

    random_forest_model(vector_x_train_transformed,vector_y_train,vector_x_test_transformed,vector_y_test)

In [65]:
vectorizer_model()

Percent variance captured by all components: 25.4197241354
Training set score: 0.973125

Test set score: 0.8171875
[ 0.81443299  0.81724461  0.80863039]


In [90]:
from sklearn.linear_model import LogisticRegression

In [95]:
#Random forest model function
def logistic_model(x_train,y_train,x_test,y_test):
    lr = LogisticRegression()
    train = lr.fit(x_train, y_train)
    print(x_train.shape, y_train.shape)
    print('Training set score:', lr.score(x_train, y_train))
    print('\nTest set score:', lr.score(x_test, x_test))